<a href="https://colab.research.google.com/github/SumitSonar/Pothole-Detection/blob/main/validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Import necessary libraries
import torch
import torchvision.transforms as transforms
from torchvision import models
import os
from PIL import Image

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths
model_path = '/content/drive/MyDrive/pothole_detection_model.pth'  # Change this path
test_images_path = '/content/drive/MyDrive/sample'  # Change this path

# Define the model architecture
class PotholeDetector(torch.nn.Module):
    def __init__(self):
        super(PotholeDetector, self).__init__()
        self.model = models.resnet18(pretrained=False)  # Change to the architecture you used
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, 2)  # Adjust to your number of classes

    def forward(self, x):
        return self.model(x)

# Instantiate the model
model = PotholeDetector()

# Load the model weights with strict=False
try:
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')), strict=False)  # Load with strict=False
    model.eval()  # Set the model to evaluation mode
except RuntimeError as e:
    print(f"Error loading model weights: {e}")

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input size
    transforms.ToTensor(),
])

# Function to evaluate the model on test images
def evaluate_model(test_images_path):
    correct = 0
    total = 0
    total_images = 0  # Total number of images in the validation set

    # Loop through all images in the test directory
    for filename in os.listdir(test_images_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Add more formats if needed
            img_path = os.path.join(test_images_path, filename)
            img = Image.open(img_path).convert('RGB')  # Open the image
            img_tensor = transform(img).unsqueeze(0)  # Transform and add batch dimension

            # Make prediction
            with torch.no_grad():
                outputs = model(img_tensor)
                _, predicted = torch.max(outputs.data, 1)

            # Check if the image has a pothole
            if 'pothole' in filename:  # Change according to your labeling convention
                pothole_count += 1

            # Assuming the labels are in the filename: e.g., 'pothole_1.jpg', 'no_pothole_2.jpg'
            label = 0 if 'pothole' in filename else 1  # Change according to your labeling convention

            total += 1
            correct += (predicted.item() == label)

            total_images += 1  # Increment the total image count

    accuracy = correct / total * 100 if total > 0 else 0
    print(f'Accuracy of the model on test images: {accuracy:.2f}%')
    print(f'Total images in validation set: {total_images}')

# Run evaluation
evaluate_model(test_images_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-36-4ee658073b28>:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')), strict=Fals

Accuracy of the model on test images: 83.87%
Total images in validation set: 31
